# Phase 2: 전처리 파이프라인

이 노트북에서 배울 내용:
- Voxel Grid 다운샘플링
- 노이즈 제거 (Statistical Outlier Removal)
- 법선 벡터 추정
- 전처리 파이프라인

In [1]:
import sys
sys.path.append('..')

import numpy as np
import open3d as o3d

from src.data import PointCloudLoader
from src.preprocessing import Preprocessor
from src.utils import Visualizer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 1. 노이즈가 있는 샘플 데이터 생성

In [2]:
loader = PointCloudLoader()
preprocessor = Preprocessor()

# 노이즈 포함 씬 생성
scene = loader.create_sample_scene(add_noise=True)
print(f"원본: {len(scene.points):,}개 포인트")

Visualizer.show(scene, "Original (with noise)")

샘플 씬 생성: 5,316개 포인트
원본: 5,316개 포인트


## 2. Voxel Grid 다운샘플링

공간을 작은 큐브(voxel)로 나누고, 각 큐브 안의 점들을 하나의 대표점으로 대체합니다.

**장점**: 데이터 양 감소, 계산 속도 향상, 균일한 점 분포

**파라미터**: `voxel_size` - 클수록 점이 적어짐

In [3]:
# 다양한 voxel size 비교
for voxel_size in [0.02, 0.05, 0.1, 0.2]:
    down = preprocessor.downsample(scene, voxel_size=voxel_size)
    print(f"voxel_size={voxel_size}: {len(down.points):,}개 포인트")

다운샘플링: 5,316 → 5,228 (98.3%)
voxel_size=0.02: 5,228개 포인트
다운샘플링: 5,316 → 4,664 (87.7%)
voxel_size=0.05: 4,664개 포인트
다운샘플링: 5,316 → 3,160 (59.4%)
voxel_size=0.1: 3,160개 포인트
다운샘플링: 5,316 → 1,286 (24.2%)
voxel_size=0.2: 1,286개 포인트


In [4]:
# 시각적 비교
down_small = preprocessor.downsample(scene, voxel_size=0.03)
down_large = preprocessor.downsample(scene, voxel_size=0.1)

Visualizer.show_comparison(down_small, down_large, "voxel=0.03", "voxel=0.1")

다운샘플링: 5,316 → 5,109 (96.1%)
다운샘플링: 5,316 → 3,160 (59.4%)
좌측: voxel=0.03 (5,109개)
우측: voxel=0.1 (3,160개)


## 3. 노이즈 제거

### Statistical Outlier Removal

각 점에 대해 k개의 이웃과의 평균 거리를 계산합니다.
전체 평균에서 표준편차의 `std_ratio`배 이상 벗어난 점을 제거합니다.

**파라미터**:
- `nb_neighbors`: 이웃 점 개수 (클수록 정교함)
- `std_ratio`: 허용 범위 (클수록 관대함)

In [5]:
# 먼저 다운샘플링
scene_down = preprocessor.downsample(scene, voxel_size=0.05)

# 노이즈 제거
scene_clean, inlier_idx = preprocessor.remove_outliers(
    scene_down,
    nb_neighbors=20,
    std_ratio=2.0
)

다운샘플링: 5,316 → 4,664 (87.7%)
노이즈 제거: 80개 이상치 제거 (1.7%)


In [6]:
# 인라이어(정상)/아웃라이어(노이즈) 시각화
outlier_idx = np.setdiff1d(range(len(scene_down.points)), inlier_idx)

inlier_cloud = scene_down.select_by_index(inlier_idx)
outlier_cloud = scene_down.select_by_index(outlier_idx)
outlier_cloud.paint_uniform_color([1, 0, 0])  # 빨간색 = 제거됨

print(f"인라이어: {len(inlier_cloud.points):,}개")
print(f"아웃라이어: {len(outlier_cloud.points):,}개")

Visualizer.show_multiple([inlier_cloud, outlier_cloud], "Inliers (gray) + Outliers (red)")

인라이어: 4,584개
아웃라이어: 80개


## 4. 법선 벡터 추정

각 점의 국소 표면 방향을 나타내는 단위 벡터입니다.

**사용처**:
- Surface Reconstruction
- 특징 추출 (FPFH 등)
- 렌더링

In [7]:
# 법선 추정
scene_with_normals = preprocessor.estimate_normals(scene_clean)

print(f"법선 벡터 수: {len(scene_with_normals.normals)}")

법선 추정 완료: 4,584개 법선
법선 벡터 수: 4584


In [8]:
# 법선 시각화 (선으로 표시)
# Open3D 뷰어에서 'N' 키를 누르면 법선 표시 토글
o3d.visualization.draw_geometries(
    [scene_with_normals],
    window_name="Normals (press N to toggle)",
    point_show_normal=True
)

## 5. 전체 파이프라인

In [9]:
# 원본 데이터 다시 로드
scene_original = loader.create_sample_scene(add_noise=True)

# 전체 파이프라인 실행
scene_processed = preprocessor.full_pipeline(
    scene_original,
    voxel_size=0.05,
    remove_noise=True,
    estimate_normals=True
)

샘플 씬 생성: 5,316개 포인트
전처리 파이프라인 시작
다운샘플링: 5,316 → 4,753 (89.4%)
노이즈 제거: 86개 이상치 제거 (1.8%)
법선 추정 완료: 4,667개 법선
전처리 완료: 최종 4,667개 포인트


In [10]:
# 전후 비교
print(f"원본: {len(scene_original.points):,}개")
print(f"처리 후: {len(scene_processed.points):,}개")

Visualizer.show_comparison(scene_original, scene_processed, "원본", "전처리 후")

원본: 5,316개
처리 후: 4,667개
좌측: 원본 (5,316개)
우측: 전처리 후 (4,667개)
